# Testing Random Forest Model on Hold-out Data

In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-aspenjkmorgan')

In [ ]:
import pandas as pd

### Import held out data

In [ ]:
# bring in the last 10 percent of data for generalizing
testing = ee.FeatureCollection('users/aspenjkmorgan/k_folds/test_v2')
testing.first().getInfo()

{'type': 'Feature',
 'geometry': {'type': 'Point', 'coordinates': [-114.189259, 48.363709]},
 'id': '00000000000000000000',
 'properties': {'DPT': 7.049996852874756,
  'PBLH': 176.98858642578125,
  'PRES': 91151.5,
  'RH': 45.5619720135636,
  'TMP': 19.07999610900879,
  'WDIR': 157.40000915527344,
  'WIND': 2.009999990463257,
  'aod': 0.0965,
  'pm25': 8.4275,
  'random': 0.7443328619811324,
  'station': 'Flathead Valley',
  'system:time_start': 1335124800000}}

### Use best split of data for training the model



In [ ]:
# best fold of training data
training = ee.FeatureCollection('users/aspenjkmorgan/k_folds/train1_v2')
training.first().getInfo()

{'type': 'Feature',
 'geometry': {'type': 'Point', 'coordinates': [-114.189259, 48.363709]},
 'id': '00000000000000000000',
 'properties': {'DPT': -0.5499938726425171,
  'PBLH': 2201.431640625,
  'PRES': 91381.5,
  'RH': 18.980433465534684,
  'TMP': 24.62998390197754,
  'WDIR': 275.0500183105469,
  'WIND': 0.6549999713897705,
  'aod': 0.0965,
  'pm25': 8.4765,
  'random': 0.003841988323326273,
  'station': 'Flathead Valley',
  'system:time_start': 1337025600000}}

### Helper functions

In [ ]:
# Compute the difference between computed pm25 and actual
def difference(feature):
  diff = ee.Number(feature.get('pm_cal')) \
    .subtract(ee.Number(feature.get('pm25')))
  # Return the feature with the squared difference set to the 'diff' property.
  return feature.set('sq_diff', diff.pow(2))

def getTss(feature):
  buf = ee.Number(feature.get('pm25')).subtract(ee.Number(mean_actual))
  return feature.set('tss', buf.pow(2))

### Use best hyperparameters in model



In [ ]:
# best estimator, fit to training data
rf_model = ee.Classifier.smileRandomForest(**{
    'numberOfTrees': 100,
    'variablesPerSplit': 3,
    'minLeafPopulation': 2,
    'bagFraction': 0.5,
    'maxNodes': None,
    'seed': 0}) \
    .setOutputMode('REGRESSION')\
    .train(**{
        'features': training,
        'classProperty': 'pm25',
        'inputProperties': ['PRES', 'aod', 'RH', 'PBLH', 'DPT', 'WIND', 'WDIR', 'TMP']
})

In [ ]:
# save model to assets for app
# (not working with app for some reason)
ee.batch.Export.classifier.toAsset(rf_model,
                                   'Saved random forest',
                                   'users/aspenjkmorgan/random_forest_2023-12-20').start()

In [ ]:
# get predictions
testing_applied = testing.classify(rf_model, 'pm_cal')

In [ ]:
testing_applied.first().getInfo()

{'type': 'Feature',
 'geometry': {'type': 'Point', 'coordinates': [-114.189259, 48.363709]},
 'id': '00000000000000000000',
 'properties': {'DPT': 7.049996852874756,
  'PBLH': 176.98858642578125,
  'PRES': 91151.5,
  'RH': 45.5619720135636,
  'TMP': 19.07999610900879,
  'WDIR': 157.40000915527344,
  'WIND': 2.009999990463257,
  'aod': 0.0965,
  'pm25': 8.4275,
  'pm_cal': 6.43877822227833,
  'random': 0.7443328619811324,
  'station': 'Flathead Valley',
  'system:time_start': 1335124800000}}

### Calculate Metrics (RMSE and $R^2$)



In [ ]:
mean_actual = testing_applied.aggregate_mean('pm25')
testing_applied = testing_applied.map(difference)
testing_applied = testing_applied.map(getTss)

# RMSE for validation data
testing_rmse = ee.Number(testing_applied.reduceColumns(ee.Reducer.mean(), ['sq_diff']).get('mean')).sqrt()

In [ ]:
print('RMSE: ' + str(testing_rmse.getInfo()))

RMSE: 10.267555130354243


In [ ]:
# get R2
rss = ee.Number(testing_applied.reduceColumns(ee.Reducer.sum(), ['sq_diff']).get('sum'))
tss = ee.Number(testing_applied.reduceColumns(ee.Reducer.sum(), ['tss']).get('sum'))
r2 = ee.Number(1).subtract(rss.divide(tss))

In [ ]:
print('R2: ' + str(r2.getInfo()))

R2: 0.5732414047367722


In [ ]:
# calculate feature importance
importance = pd.DataFrame(rf_model.explain().get('importance').getInfo(), index=[0])
total = importance.sum(axis=1)

def normalize(col):
  return round((col / total)*100, 3)

feature_importance = importance.apply(normalize)
feature_importance

,DPT,PBLH,PRES,RH,TMP,WDIR,WIND,aod
0,12.041,9.069,9.748,10.864,11.96,7.564,12.092,26.662


In [ ]:
# out of bag score
# out of bag score
rf_model.explain().get('outOfBagErrorEstimate').getInfo()

10.272522573195483